In [1]:
# Python 3.x - Stolen shamelessly from https://github.com/norvig/pytudes
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = 'inputs/day{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("Go get it bellend")
    


def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

def flatten(l):
    "Flatten a list of lists"
    return [item for sublist in l for item in sublist]

# Day 1:
## Part 1 
Given a number n, find the sum of digits equal to the next digit, where the final digit's successor is the first (i.e. given 1123441, return 1 + 4 + 1 = 6)
## Part 2
As above, but instead of sum digits equal to their successor, sum digits equal to their antipode (i.e. the digit m digits away, where 2*m is the length of n.

In [17]:
one = Input(1).read()
def skip_sum(skip, one = one):
    total = 0
    for i in range(len(one)):
        try:
            total += int(one[i]) if one[i] == one[i + skip] else 0
        except IndexError:
            comp = (i + skip) % len(one)
            total += int(one[i]) if one[i] == one[comp] else 0
    return total
print('The part 1 total is: {}'.format(skip_sum(1)))
print('The part 2 total is: {}'.format(skip_sum(len(one)//2)))

The part 1 total is: 1393
The part 2 total is: 1292


In [25]:
#A numpy sol
try:
    ipt = Input('1_np').read()
except:
    !sed 's/[0-9]/& /g' <inputs/day1.txt >inputs/day1_np.txt
    ipt = Input('1_np').read()
    
def np_skip_sum(skip, ipt = ipt):
    x = np.fromstring(ipt.strip(), dtype = int, sep = ' ')
    mask = x - np.roll(x, skip)
    out = np.dot(x, mask==0)
    return out
print('The part 1 numpy total is: {}'.format(np_skip_sum(1)))
print('The part 2 numpy total is: {}'.format(np_skip_sum(len(ipt)//4))) #The sed command adds a space after every digit

The part 1 numpy total is: 1393
The part 2 numpy total is: 1292


# Day 2
## Part 1
Checksum of spreadsheet


In [5]:
two = Input(2).read()
two
len(two.split('\n')[0].split('\t'))

16

In [8]:
def get_shape(string):
    n = string.split('\n')
    m = n[0].split('\t')
    return (len(n), len(m))

In [9]:
get_shape(two)

(16, 16)

In [16]:
two = np.fromstring(two, dtype = int, sep = '\t').reshape(get_shape(two))

In [23]:
np.sum(np.amax(two, axis =1) - np.amin(two, axis = 1))

30994

In [29]:
total = 0
for k in range(16):
    for i in range(16):
        for j in range(i + 1, 16):
            if two[k][j] % two[k][i] == 0:
                total += two[k][j] // two[k][i]
                continue
        if two[k][j] % two[k][i] == 0:
            continue
total            

233

# Day 3
## Part 1
Manhattan distance in spiral grid

In [2]:
i = 312051
math.ceil(math.sqrt(i))

559

In [3]:
def find_square(i): 
    '''return bottom-right coordinate of outer square containing i'''
    sq = math.ceil(math.sqrt(i))
    sq -= 1 if (sq % 2 == 1) else 0
    sq //= 2
    return (sq, sq, (2*sq + 1)**2 - i)
def _fs(i):
    sq = math.ceil(math.sqrt(i))
    sq += 1 if (sq % 2 == 0) else 0
    return sq**2

In [4]:
def walk(start, dist):
    '''Coordinate of end after walking dist counterclockwise from start'''
    step_size = X(start) + 1 // 2
    return X(start) + (dist % step_size)

In [5]:
walk(find_square(i), find_square(i)[2])

430

## Part 2

In [5]:
y = np.zeros(21**2).reshape(21,21)
y[10,10] = 1

In [2]:
def sub_(n,m):
    return y[n-1:n+2, m-1:m+2]

def update_y(n,m):
    y[n,m] = np.sum(sub_(n,m))

In [3]:
sqVals = [0]*(21**2)
sqVals[0] = 1
def sq_val(sq, outer_square):
    

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

# Day 4
## Part 1
Given list of passphrases, find number of passphrases without repeated words

In [128]:
four = Input(4).read()

In [123]:
#Just count occurance of each word in each passphrase, then count number of passphrases with highest counts equal to 1
pt1 = Counter([Counter(line.split()).most_common(1)[0][1] for line in four.split('\n')])
pt1[1]

455

In [124]:
def is_anagram(word1, word2):
    """An anagram tester I was going to use but then didn't"""
    if not len(word1) == len(word2):
        return False
    c1 = Counter(word1)
    c2 = Counter(word2)
    for k, v in c1.items():
        if not (v == c2[k]):
            return False
    return True
is_anagram('hello', 'llohe'), is_anagram('Tuesday', 'Mondays')

(True, False)

## Part 2
Given same list of passphrases, return number of phrases without anagrams in them (i.e. all words are pairwise _not_ anagrams of each other)

In [131]:
pwords = four.split('\n')

In [129]:
def sort_line(line):
    return [''.join(sorted(word)) for word in line.split()]

In [132]:
sorted_pwords = [sort_line(line) for line in pwords]

In [135]:
#After rearranging each word into alphabetical order, can just proceed as above.
pt2 = Counter([Counter(line).most_common(1)[0][1] for line in sorted_pwords])
pt2[1]

186

# Day 5
## Part 1
Escape instruction set

In [32]:
five = Input(5).read().split('\n')

In [38]:
index = 0
moves = 0
five_copy = [int(l) for l in five]
while index > -1:
    try:
        new_index = index + five_copy[index]
    except IndexError:
        print(moves)
        break
    five_copy[index] += 1
    index = new_index
    moves += 1
print(moves)

356945
356945


## Part 2


In [42]:
%prun
def func(five = five):
    index = 0
    moves = 0
    five_copy = [int(l) for l in five]
    while index > -1:
        try:
            new_index = index + five_copy[index]
        except IndexError:
            return moves
            break
        five_copy[index] += 1 if five_copy[index] < 3 else -1
        index = new_index
        moves += 1
    return moves
func(five)

28372145

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)

# Day 6
## Part 1
Memory redistribution

In [28]:
six = Input(6).read()
six_array = [int(s) for s in six.split('\t')]

In [4]:
six_array

[5, 1, 10, 0, 1, 7, 13, 14, 3, 12, 8, 10, 7, 12, 0, 6]

In [14]:
def redist(lst):
    l = len(lst)
    m = max(lst)
    i = lst.index(m)
    lst[i] = 0
    while m > 0:
        i += 1
        i %= l
        lst[i] += 1
        m -= 1
    return lst

In [31]:
six = Input(6).read()
six_array = [int(s) for s in six.split('\t')]
seen_arrays = set()
seen_arrays.add('_'.join([str(i) for i in six_array]))
steps = 0
m = six_array
while True:
    m = redist(m)
    new_array = '_'.join([str(i) for i in m])
    steps += 1
    if new_array in seen_arrays:
        print(steps)
        break
    else:
        seen_arrays.add(new_array)
    if steps > 10000:
        break

5042


In [32]:
six = Input(6).read()
six_array = [int(s) for s in six.split('\t')]
seen_arrays = list()
seen_arrays.append('_'.join([str(i) for i in six_array]))
steps = 0
m = six_array
while True:
    m = redist(m)
    new_array = '_'.join([str(i) for i in m])
    steps += 1
    if new_array in seen_arrays:
        print(steps - seen_arrays.index(new_array))
        break
    else:
        seen_arrays.append(new_array)
    if steps > 10000:
        break

1086


# Day 7
## Part 1
Find base name

In [2]:
seven = Input(7).read().split('\n')

In [4]:
names = [s.split()[0] for s in seven]
mounted_names = flatten([n.split(', ') for n in [m.split('-> ')[1] for m in seven if re.search('->', m)]])

In [5]:
root = set(names)- set(mounted_names)

next(iter(root))

'qibuqqg'

## Part 2

In [6]:
def weight(program):
    line = [l for l in seven if l.startswith(program)][0]
    children = get_children(program)
    w = int(re.search('\d+', line).group())
    weights = [w]
    for child in children:
        cw = weight(child)[0]
        w += cw
        weights.append(cw)
    return [w, weights]

def get_children(program):
    children = []
    line = [l for l in seven if l.startswith(program)][0]
    if re.search('->', line):
        children = line.split('-> ')[1].split(', ')
    return children

def is_balanced(program):
    children = get_children(program)
    child_weights = dict(zip(children, [weight(c)[0] for c in children]))
    c = Counter(child_weights.values())
    if len(c) == 1:
        return None
    odd_one_out = min(c, key = c.get)
    return [k for k,v in child_weights.items() if v == odd_one_out][0]

In [7]:
prog = next(iter(root))

while is_balanced(prog):
    print('{} is unbalanced'.format(prog))
    children = get_children(prog)
    print("Its children are:")
    for c in children:
        print("\t{}, whose total weight is {}".format(c, weight(c)[0]))
    prog = is_balanced(prog)
    print('{} is the odd one out'.format(prog))
    print('\n')
    
child_weights = Counter([weight(c)[0] for c in children])
weight_diff = max(child_weights) - min(child_weights)

print('{} is balanced. It\'s own weight is {}.\nIt should be {}.'
      .format(prog, weight(prog)[1][0], weight(prog)[1][0] - weight_diff))

qibuqqg is unbalanced
Its children are:
	oxxhpbs, whose total weight is 114292
	tiekffc, whose total weight is 114292
	nxgtet, whose total weight is 114292
	nidfqb, whose total weight is 114292
	cctmov, whose total weight is 114292
	wpwznmz, whose total weight is 114292
	dwggjb, whose total weight is 114299
dwggjb is the odd one out


dwggjb is unbalanced
Its children are:
	ikplxqu, whose total weight is 8278
	wacoc, whose total weight is 8278
	wknuyhc, whose total weight is 8285
wknuyhc is the odd one out


wknuyhc is unbalanced
Its children are:
	xznmwt, whose total weight is 1640
	mhbquyl, whose total weight is 1640
	alfjsh, whose total weight is 1640
	pxfelm, whose total weight is 1640
	egbzge, whose total weight is 1647
egbzge is the odd one out


egbzge is balanced. It's own weight is 1086.
It should be 1079.


# Day 8
## Part 1
Something something registers

In [51]:
eight = Input(8).read().split('\n')

In [53]:
def condition(line):
    cond = line.split('if ')[1]
    cond = re.sub('([a-z]+)', 'registers["\\1"]', cond)
    neg = re.search('!', cond)
    if neg:
        cond = 'not ' + re.sub('!', '=', cond)
    
    return eval(cond)

In [57]:
registers = {l.split()[0]: 0 for l in eight}
for line in eight:
    if condition(line):
        line = line.split()
        reg = line[0]
        instr = line[1]
        incr = int(line[2])
        if instr == 'inc':
            registers[reg] += incr
        else:
            registers[reg] -= incr
print(max(registers.values()))

3089


## Part 2

In [58]:
registers = {l.split()[0]: 0 for l in eight}
max_value = 0
for line in eight:
    if condition(line):
        line = line.split()
        reg = line[0]
        instr = line[1]
        incr = int(line[2])
        if instr == 'inc':
            registers[reg] += incr
        else:
            registers[reg] -= incr
        m = max(registers.values())
        if m > max_value:
            max_value = m
print(max_value)

5391


# Day 9
## Part 1
Data stream, nested brackets

In [62]:
nine = Input(9).read()

In [63]:
open_groups = 0
current_group_score = 0
current_score = 0
is_garbage = False
iter_nine = iter(nine)
for char in iter_nine:
    if char == '!':
        next(islice(iter_nine,0,1))
        continue
    if char == '<':
        is_garbage = True
    if char == '>':
        is_garbage = False
    if not is_garbage:
        if char == '{':
            open_groups += 1
            current_group_score +=1
        if char == '}':
            current_score += current_group_score
            open_groups -= 1
            current_group_score -= 1
print(current_score)

16869


## Part 2

In [60]:
current_score = 0
is_garbage = False
iter_nine = iter(nine)
for char in iter_nine:
    if char == '!':
        next(islice(iter_nine,0,1))
        continue
    if char == '<':
        is_garbage = True
    if char == '>':
        is_garbage = False
        current_score -= 1 #Because we counted the opening angle bracket
    if is_garbage:
        current_score += 1
print(current_score)

7284


# Day 10
## Part 1

In [28]:
ten = Input(10).read()
ten

'192,69,168,160,78,1,166,28,0,83,198,2,254,255,41,12'

In [29]:
def list_update(index, length, lst):
    l = lst.copy()
    _len = len(l)
    y = None
    if index + length > _len:
        rot = (index + length) % _len
        l = np.roll(l, -rot)
        y = l[-length::]
        l[-length::] = y[::-1]
        l = np.roll(l, rot)
    else:
        y = l[index:index+length:]
        l[index:index+length:] = y[::-1]
    return l

In [30]:
_list = np.array(np.arange(256))
input_lengths = map(lambda x: int(x), ten.split(','))
step_size = -1
index = 0
l = len(_list)
for _ in input_lengths:
    _list = list_update(index, _, _list)
    step_size += 1
    index = (index + _ + step_size) % l
print(np.prod(_list[:2]))

48705


## Part 2

In [31]:
def to_ascii(string):
    suffix = [17, 31, 73, 47, 23]
    out = list(map(lambda x: ord(x), string))
    out.extend(suffix)
    return out

In [32]:
def dense_hash(narray, block_size):
    if not len(narray) % block_size == 0:
        return 'Block_size must evenly divide array length'
    rows, cols = [len(narray) // block_size ,block_size]
    out = narray.reshape(rows, cols)
    return np.bitwise_xor.reduce(out, axis = 1)

In [33]:
_list = np.array(np.arange(256))
input_lengths = to_ascii(ten)
step_size = -1
index = 0
l = len(_list)

for i in range(64):
    for _ in input_lengths:
        _list = list_update(index, _, _list)
        step_size += 1
        index = (index + _ + step_size) % l

dense_hash_array = dense_hash(_list, 16)
hex_array = np.vectorize(hex)(dense_hash_array)
leading_zero = lambda x: x[-2:] if len(x) == 4 else '0' + x[-1]
lz = np.vectorize(leading_zero)

print(''.join(lz(hex_array)))

1c46642b6f2bc21db2a2149d0aeeae5d


In [35]:
def knot_hash(ipt):
    _list = np.array(np.arange(256))
    input_lengths = to_ascii(ipt)
    step_size = -1
    index = 0
    l = len(_list)

    for i in range(64):
        for _ in input_lengths:
            _list = list_update(index, _, _list)
            step_size += 1
            index = (index + _ + step_size) % l
    dense_hash_array = dense_hash(_list, 16)
    hex_array = np.vectorize(hex)(dense_hash_array)
    leading_zero = lambda x: x[-2:] if len(x) == 4 else '0' + x[-1]
    lz = np.vectorize(leading_zero)
    return ''.join(lz(hex_array))
print(knot_hash(ten))

1c46642b6f2bc21db2a2149d0aeeae5d


# Day 11
## Part 1
Distance in a hex grid

In [2]:
eleven = Input(11).read()

In [ ]:
def current_dist(coord):
    x = np.sum(coord >= 0)
    if x == 1:
        return max(np.abs(coord))
    else:
        return abs(np.sum(coord))

In [54]:
steps = Counter(eleven.split(','))
directions = dict(zip(['n', 's', 'nw', 'sw', 'se', 'ne'], [np.array(p) for p in permutations([0,1,-1], 2)]))
current_coord = np.array((0,0))

In [55]:
for d, vec in directions.items():
    current_coord += vec*steps[d]
print(current_dist(current_coord))

664


## Part 2

In [53]:
current_coord = np.array((0,0))
max_dist = 0
for step in eleven.split(','):
    current_coord += directions[step]
    dist = current_dist(current_coord)
    if dist > max_dist:
        max_dist = dist
print(max_dist)

1447


# Day 12
## Part 1
Programs in a group

In [3]:
twelve = Input(12).read()

'0 <-> 584, 830\n1 <-> 415, 531\n2 <-> 514, 1419\n3 <-> 3\n4 <-> 1043\n5 <-> 440\n6 <-> 452, 1651, 1861\n7 <-> 368, 783, 1364\n8 '

In [4]:
list_input = twelve.split('\n')
len(list_input)

2000

In [18]:
def communicates_with(i):
    cw = list_input[i].split(' <-> ')[1].split(', ')
    return [int(c) for c in cw]

group_0 = set([0])
cands = communicates_with(0)
while cands:
    c = cands.pop()
    new_cands = communicates_with(c)
    group_0.add(c)
    cands.extend([nc for nc in new_cands if nc not in group_0])
print(len(group_0))

113


## Part 2

In [22]:
def get_group(i):
    group_ = set([i])
    cands = communicates_with(i)
    while cands:
        c = cands.pop()
        new_cands = communicates_with(c)
        group_.add(c)
        cands.extend([nc for nc in new_cands if nc not in group_])
    return group_
len(get_group(1))

113

In [38]:
remaining_elts = set(range(len(list_input)))
groups = 0
while len(remaining_elts) > 0:
    x = remaining_elts.pop()
    grp_x = get_group(x)
    groups += 1
    remaining_elts -= grp_x
print(groups)

202


# Day 13
## Part 1

In [46]:
thirteen = Input(13).read().split('\n')


In [49]:
def collision(row, delay = 0):
    "Detect if scanner at top of firewall - i.e. if step % 2*(range-1) == 0, calculate severity"
    r = [int(n) for n in row.split(': ')]
    scanner_start = 1 + (delay % r[1])
    if (r[0] + delay) % (2*r[1] - 2) == 0:
        return r[0] * r[1]
    else:
        return 0

In [50]:
sum([collision(row,1 ) for row in thirteen])

2

## Part 2

In [103]:
def scanner_pos(l, ps):
    "Get position of scanner at of length l at time ps"
    if l == 1:
        return 0
    x = ps % (2*l - 2)
    if x <= (l-1):
        return x
    else:
        return l - 1 - (x % (l-1))
[scanner_pos(3,s) for s in range(15)]

[0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 2]

In [131]:
%%time
delay = 0
initial_ = [[int(n) for n in row.split(': ')] for row in thirteen]
#Hella inefficient
while 2*delay < 4000000:
    pos = list(map(lambda x: scanner_pos(x[1], 2*delay + x[0]), initial_))
    if all(pos):
        print(2*delay)
        break
    else:
        delay += 1

3933124
CPU times: user 39 s, sys: 37.3 ms, total: 39 s
Wall time: 39 s


In [136]:
%%time
delay = 0
while 2*delay < 4000000:
    skip = False
    for _ in initial_:
        p = scanner_pos(_[1], 2*delay + _[0])
        if p == 0:
            skip = True
            break
    if skip:
        delay += 1
    else:
        print(2*delay)
        break

3933124
CPU times: user 3.63 s, sys: 9.28 ms, total: 3.64 s
Wall time: 3.67 s


# Day 14
## Part 1

In [43]:
def to_bin(x):
    h = knot_hash(x)
    pad_bin = lambda b: b[2:] if len(b) == 6 else '0'*(6 - len(b)) + b[2:]
    return ''.join(map(pad_bin, [bin(int(x, 16)) for x in h]))
# hex(ord('0'))
to_bin('a0c2017'), knot_hash('a0c2017')
# [to_bin(x) for x in 'a0c2017']

('0b11100001111011011010100010101000000110110111111001110010011101101111011000111000011111111010110010100011011010000010100110010010',
 'e1eda8a81b7e7276f6387faca3682992')

In [46]:
[bin(int(x, 16)) for x in knot_hash('a0c2017')]
pad_bin = lambda b: b[2:] if len(b) == 6 else '0'*(6 - len(b)) + b[2:]
''.join(map(pad_bin, [bin(int(x, 16)) for x in 'a0c2017']))

'1010000011000010000000010111'

In [48]:
x = 'flqrgnkx'
for i in range(8):
    y = x + '-' + str(i)
    h = to_bin(y)
    print(h[:8])

0b110101
0b101010
0b101011
0b101011
0b110100
0b110010
0b100010
0b110101


# Day 15
## Part 1

In [2]:
fifteen = Input(15).read()
print(fifteen)

Generator A starts with 289
Generator B starts with 629


In [3]:
def gen_A(start):
    return (start*16807) % 2147483647

def gen_B(start):
    return (start*48271) % 2147483647

def compare(*args):
    x = args[0] ^ args[1]
    return x % 2**16
compare((1,2))

In [37]:
A_prev = 289
B_prev = 629
pair_count = 0
for i in range(40000000):
    A_prev = gen_A(A_prev)
    B_prev = gen_B(B_prev)
    if compare(A_prev, B_prev) == 0:
        pair_count += 1
print(pair_count)

638


## Part 2
    

In [53]:
from queue import Queue
A_prev = 289
B_prev = 629
A_queue = Queue()
B_queue = Queue()
pair_count = 0
for i in range(50):
    A_prev = gen_A(A_prev)
    B_prev = gen_B(B_prev)
    if A_prev % 4 == 0:
        A_queue.put(A_prev)
    if B_prev % 8 == 0:
        B_queue.put(B_prev)
    if not (A_queue.empty() or B_queue.empty()):
        if compare(A_queue.get(), B_queue.get()) == 0:
            pair_count += 1
print(pair_count)

0


In [54]:
A_queue.qsize(), B_queue.qsize()

(0, 2)

In [81]:
from queue import Queue
A_prev = 289
B_prev = 629
A_queue = Queue()
B_queue = Queue()
pair_count = 0
pairs_seen = 0
while pairs_seen <= 5000000:
    A_prev = gen_A(A_prev)
    B_prev = gen_B(B_prev)
    if A_prev % 4 == 0:
        A_queue.put(A_prev)
    if B_prev % 8 == 0:
        B_queue.put(B_prev)
    if not (A_queue.empty() or B_queue.empty()):
        pairs_seen += 1
        a = A_queue.get()
        b = B_queue.get()
        #print(a, b)
        if compare(a, b) == 0:
            #print(a, b, i)
            pair_count += 1
print(pair_count)

343


# Day 16
## Part 1

In [2]:
sixteen = Input(16).read()

In [98]:
moves = sixteen.split(',')

In [99]:
posns = 'a b c d e f g h i j k l m n o p'.split()
l = len(posns)
for move in moves:
    m = move[0]
    if m == 'x':
        inds = [int(c) for c in move[1:].split('/')]
        posns[inds[0]], posns[inds[1]] = [posns[x] for x in inds[::-1]]
    elif m == 's':
        slc = int(move[1:])
        posns = posns[-slc:] + posns[:l-slc]
    else:
        inds = [posns.index(p) for p in move[1:].split('/')]
        posns[inds[0]], posns[inds[1]] = [posns[x] for x in inds[::-1]]
        
print(''.join(posns))

dcmlhejnifpokgba


## Part 2

In [100]:
posns = 'a b c d e f g h i j k l m n o p'.split()
#posns = 'a b c d e'.split()
l = len(posns)
for _ in range(500):
    for move in moves:
    #     print(move)
        m = move[0]
        if m == 'x':
            inds = [int(c) for c in move[1:].split('/')]
            posns[inds[0]], posns[inds[1]] = [posns[x] for x in inds[::-1]]
        elif m == 's':
            slc = int(move[1:])
            posns = posns[-slc:] + posns[:l-slc]
        else:
            inds = [posns.index(p) for p in move[1:].split('/')]
            posns[inds[0]], posns[inds[1]] = [posns[x] for x in inds[::-1]]
    if ''.join(posns) == 'abcdefghijklmnop':
        print('Order is {}.\nOnly need to check the arrangement after {} rounds.'.format(_ + 1, 10**9 %( _ + 1)))
        
        break

Order is 24.
Only need to check the arrangement after 16 rounds.


In [101]:
posns = 'a b c d e f g h i j k l m n o p'.split()
l = len(posns)
for _ in range(16):
    for move in moves:
        m = move[0]
        if m == 'x':
            inds = [int(c) for c in move[1:].split('/')]
            posns[inds[0]], posns[inds[1]] = [posns[x] for x in inds[::-1]]
        elif m == 's':
            slc = int(move[1:])
            posns = posns[-slc:] + posns[:l-slc]
        else:
            inds = [posns.index(p) for p in move[1:].split('/')]
            posns[inds[0]], posns[inds[1]] = [posns[x] for x in inds[::-1]]

print(''.join(posns))

ifocbejpdnklamhg


# Day 17
## Part 1 and 2

In [8]:
def _insert(lst, el, index):
    "Insert el into lst at index"
    return lst[:index] + [el] + lst[index:]

In [13]:
%%time
# from numba import jit
# @jit
def day17(step_size):
    lst = [0]
    current_index = 0
    ind_1 = []
    for i in range(1, 50000000):
        current_index = (current_index + step_size + 1) % i
        if current_index == 0:
            ind_1.append(i)
        if i < 2018:
            lst = _insert(lst, i, current_index + 1)
            p1_index = current_index + 2
    print('Part 1 answer is {}.\nPart 2 answer is {}.'.format(lst[p1_index], ind_1[-1]))

day17(314)

Part 1 answer is 355.
Part 2 answer is 6154117.
CPU times: user 7.1 s, sys: 5.72 ms, total: 7.1 s
Wall time: 7.11 s


# Day 18
## Part 1

In [14]:
eighteen = Input(18).read().split('\n')
eighteen

['set i 31',
 'set a 1',
 'mul p 17',
 'jgz p p',
 'mul a 2',
 'add i -1',
 'jgz i -2',
 'add a -1',
 'set i 127',
 'set p 316',
 'mul p 8505',
 'mod p a',
 'mul p 129749',
 'add p 12345',
 'mod p a',
 'set b p',
 'mod b 10000',
 'snd b',
 'add i -1',
 'jgz i -9',
 'jgz a 3',
 'rcv b',
 'jgz b -1',
 'set f 0',
 'set i 126',
 'rcv a',
 'rcv b',
 'set p a',
 'mul p -1',
 'add p b',
 'jgz p 4',
 'snd a',
 'set a b',
 'jgz 1 3',
 'snd b',
 'set f 1',
 'add i -1',
 'jgz i -11',
 'snd a',
 'jgz f -16',
 'jgz a -19']

In [37]:
test = Input('xx').read().split('\n')
registers = {ins.split()[1] : 0 for ins in eighteen}
j = 0
sounds_played = []
i = 0
while i<10000:
    instr = eighteen[j].split()
#     print(instr)
    if instr[0] == 'set':
        try:
            registers[instr[1]] = int(instr[2])
        except ValueError:
            registers[instr[1]] = registers[instr[2]]
    elif instr[0] == 'snd':
        sounds_played.append(registers[instr[1]])
    elif instr[0] == 'add':
        try:
            registers[instr[1]] += int(instr[2])
        except ValueError:
            registers[instr[1]] += registers[instr[2]]
    elif instr[0] == 'mul':
        try:
            registers[instr[1]] *= int(instr[2])
        except ValueError:
            registers[instr[1]] *= registers[instr[2]]
    elif instr[0] == 'mod':
        try:
            registers[instr[1]] %= int(instr[2])
        except ValueError:
            registers[instr[1]] %= registers[instr[2]]
    elif instr[0] == 'jgz':
        if registers[instr[1]] > 0:
            try:
                j += int(instr[2]) - 1 #-1 because we increment j at the end of the loop every time.
            except ValueError:
                j += registers[instr[2]] - 1
    else:
        if registers[instr[1]] > 0:
            print('Last played sound had frequency {}.'.format(sounds_played[-1]))
            break
    j += 1
    i += 1

Last played sound had frequency 2951.


## Part 2

In [139]:
def _step(prog, j, structions = eighteen, sent0 = sent0, sent1 = sent1):
    registers = reg0 if prog == 0 else reg1
    sent = sent0 if prog == 0 else sent1
    rec = sent1 if prog == 0 else sent0
    other_reg = reg1 if prog == 0 else reg0
    instr = structions[j].split()
    
    if instr[0] == 'set':
        try:
            registers[instr[1]] = int(instr[2])
        except ValueError:
            registers[instr[1]] = registers[instr[2]]
    elif instr[0] == 'snd':
        sent.append(registers[instr[1]])
    elif instr[0] == 'add':
        try:
            registers[instr[1]] += int(instr[2])
        except ValueError:
            registers[instr[1]] += registers[instr[2]]
    elif instr[0] == 'mul':
        try:
            registers[instr[1]] *= int(instr[2])
        except ValueError:
            registers[instr[1]] *= registers[instr[2]]
    elif instr[0] == 'mod':
        try:
            registers[instr[1]] %= int(instr[2])
        except ValueError:
            registers[instr[1]] %= registers[instr[2]]
    elif instr[0] == 'jgz':
        if registers[instr[1]] > 0:
            try:
                j += int(instr[2]) - 1 #-1 because we increment j at the end of the loop every time.
            except ValueError:
                j += registers[instr[2]] -1
    else:
        if len(rec) > 0:
            registers[instr[1]] = rec[0]
            if prog == 0:
                sent1 = rec[1:]
            else:
                sent0 = rec[1:]
        else:
            return [False, j, instr]
    j += 1
    return [True, j, instr]

In [108]:
eighteen[25]

'rcv a'

In [140]:
i = 0
proceed0 = True
proceed1 = True
reg0 = {ins.split()[1] : 0 for ins in eighteen}
reg1 = {ins.split()[1] : 0 for ins in eighteen}
reg1['p'] = 1 
sent0 = []
sent1 = []
j0 = 0
j1 = 0
one_has_sent = 0
while i < 150:
    pr = False
    if not proceed0 and not proceed1:
        print('Terminated. Program 1 sent {} values.'.format(one_has_sent))
        break
    if proceed0 or len(sent1) > 0:
        proceed0, j0, instr0 = _step(0, j0)
    if proceed1 or len(sent0) > 0:
        print(j1, len(sent0))
        if eighteen[j1].startswith('snd'):
            one_has_sent += 1
        if eighteen[j1].startswith('rcv'):
            pr = True
            print(len(sent0))
        proceed1, j1, instr1 = _step(1, j1)
        if pr:
            print(len(sent0))
    i += 1
#     print(reg0, sent0, instr0)
#     print(reg1, sent1, instr1)
    

0 0
1 0
2 0
3 0
20 0
23 0
24 0
25 0
0
0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
26 0
0
0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0


In [144]:
one_has_sent
sent0, sent1, proceed0

([], [], True)

In [134]:
j0

0

In [57]:
reg0

{'1': 0, 'a': 1, 'b': 0, 'f': 0, 'i': 0, 'p': 0}

In [58]:
sent0

[1]

# Day 19
## Part 1

In [150]:
nineteen = Input(19).read()
Counter(nineteen)

Counter({'\n': 201,
         ' ': 24826,
         '+': 448,
         '-': 7246,
         'D': 1,
         'E': 1,
         'F': 1,
         'I': 1,
         'J': 1,
         'N': 1,
         'T': 1,
         'V': 1,
         'W': 1,
         'Z': 1,
         '|': 7871})

In [148]:
grid = nineteen.split('\n')

In [151]:
grid[0].index('|'), grid[0][13]

(13, '|')

In [ ]:
def traverse(start):
    path_char = grid[0][start]
    
    

# Day 20
## Part 1
Find particle that stays nearest (Mnhtn) to the origin

In [160]:
twenny = Input(20).read().split('\n')
twenny[:5]

['p=<-717,-4557,2578>, v=<153,21,30>, a=<-8,8,-7>',
 'p=<1639,651,-987>, v=<29,-19,129>, a=<-5,0,-6>',
 'p=<-10482,-248,-491>, v=<4,10,81>, a=<21,0,-4>',
 'p=<-6607,-2542,1338>, v=<-9,52,-106>, a=<14,2,4>',
 'p=<-4468,1178,-6474>, v=<146,44,66>, a=<0,-5,9>']

In [200]:
twenny[:10], len(twenny)
def extract_(line, ix):
    line = line.split(', ')[ix]
#     print(line)
    want = re.search('<(.*)>', line).group(1)
    #print(want)
    return [int(a) for a in want.split(',')]

p, v, a = [np.array([extract_(l, i) for l in twenny]) for i in range(3)]

In [201]:
_v = np.copy(v)
_p = np.copy(p)
for _ in range(1000):
    _p += _v
    _v += a 
print(np.argmin(np.sum(np.abs(_p), axis = 1)))

344


# Part 2

In [205]:
len(np.unique(_p))

2996

# Day 21
## Part 1

In [3]:
pat = '.#./..#/###'
rulez = Input(21).read().split('\n')

In [4]:
rulez[:3]

['../.. => ###/.##/#..', '#./.. => #.#/..#/#..', '##/.. => ###/.#./###']

In [10]:
np.array([list(l) for l in pat.split('/')])

array([['.', '#', '.'],
       ['.', '.', '#'],
       ['#', '#', '#']],
      dtype='<U1')

In [6]:
rulez

['../.. => ###/.##/#..',
 '#./.. => #.#/..#/#..',
 '##/.. => ###/.#./###',
 '.#/#. => ##./###/...',
 '##/#. => ##./###/#.#',
 '##/## => #.#/#.#/###',
 '.../.../... => #.#./#..#/#.##/#.#.',
 '#../.../... => #.##/.##./#..#/.###',
 '.#./.../... => #..#/#.#./.#../#.##',
 '##./.../... => .#../...#/####/...#',
 '#.#/.../... => ##.#/..#./...#/###.',
 '###/.../... => #.#./#..#/####/##..',
 '.#./#../... => ..##/..##/####/##.#',
 '##./#../... => #.#./#.../..../##..',
 '..#/#../... => .#.#/##.#/...#/####',
 '#.#/#../... => .#../.###/.##./##.#',
 '.##/#../... => ##../.#.#/#.../..##',
 '###/#../... => ##.#/##.#/.###/##..',
 '.../.#./... => ..#./..../##.#/#.#.',
 '#../.#./... => ..##/.#.#/..#./###.',
 '.#./.#./... => ...#/.#../.#.#/##..',
 '##./.#./... => #..#/.###/##../#.#.',
 '#.#/.#./... => ##.#/..#./.#../#..#',
 '###/.#./... => #.#./####/#..#/#...',
 '.#./##./... => ##../##.#/.###/##..',
 '##./##./... => .#../####/.##./.#..',
 '..#/##./... => ####/##.#/##.#/###.',
 '#.#/##./... => .##./#.##/##.#

# Day 24
## Part 1

In [2]:
tw4 = Input(24).read().split('\n')
tw4

['25/13',
 '4/43',
 '42/42',
 '39/40',
 '17/18',
 '30/7',
 '12/12',
 '32/28',
 '9/28',
 '1/1',
 '16/7',
 '47/43',
 '34/16',
 '39/36',
 '6/4',
 '3/2',
 '10/49',
 '46/50',
 '18/25',
 '2/23',
 '3/21',
 '5/24',
 '46/26',
 '50/19',
 '26/41',
 '1/50',
 '47/41',
 '39/50',
 '12/14',
 '11/19',
 '28/2',
 '38/47',
 '5/5',
 '38/34',
 '39/39',
 '17/34',
 '42/16',
 '32/23',
 '13/21',
 '28/6',
 '6/20',
 '1/30',
 '44/21',
 '11/28',
 '14/17',
 '33/33',
 '17/43',
 '31/13',
 '11/21',
 '31/39',
 '0/9',
 '13/50',
 '10/14',
 '16/10',
 '3/24',
 '7/0',
 '50/50']

In [5]:
[l for l in tw4 if re.search('(^0)|(/0$)',l)]

['0/9', '7/0']

In [14]:
comps = sorted([sorted([int(a) for a in l.split('/')]) for l in tw4], key = lambda x: x[0])

In [15]:
comps

[[0, 9],
 [0, 7],
 [1, 1],
 [1, 50],
 [1, 30],
 [2, 3],
 [2, 23],
 [2, 28],
 [3, 21],
 [3, 24],
 [4, 43],
 [4, 6],
 [5, 24],
 [5, 5],
 [6, 28],
 [6, 20],
 [7, 30],
 [7, 16],
 [9, 28],
 [10, 49],
 [10, 14],
 [10, 16],
 [11, 19],
 [11, 28],
 [11, 21],
 [12, 12],
 [12, 14],
 [13, 25],
 [13, 21],
 [13, 31],
 [13, 50],
 [14, 17],
 [16, 34],
 [16, 42],
 [17, 18],
 [17, 34],
 [17, 43],
 [18, 25],
 [19, 50],
 [21, 44],
 [23, 32],
 [26, 46],
 [26, 41],
 [28, 32],
 [31, 39],
 [33, 33],
 [34, 38],
 [36, 39],
 [38, 47],
 [39, 40],
 [39, 50],
 [39, 39],
 [41, 47],
 [42, 42],
 [43, 47],
 [46, 50],
 [50, 50]]

In [20]:
def candidates(comp):
    return [c for c in comps if comp[0] in c or comp[1] in c]

In [22]:
[candidates(o) for o in candidates(comps[0])]

[[[0, 9], [0, 7], [9, 28]],
 [[0, 9], [0, 7], [7, 30], [7, 16]],
 [[0, 9], [2, 28], [6, 28], [9, 28], [11, 28], [28, 32]]]